# Data set up 

## Load existing dataset

In [ ]:
import pandas as pd
df = pd.read_csv('tournesol_scores_above_20_2023-09-18.csv')

## Create dataset

Download videos scores

In [1]:
import io
import zipfile

import requests
import pandas as pd

response = requests.get("https://api.tournesol.app/exports/all")
zip_file = zipfile.ZipFile(io.BytesIO(response.content))
collective_scores = pd.read_csv(zip_file.open("collective_criteria_scores.csv"))

Requests youtube metadata : publication date, title and channel

In [2]:
from dataset import build_dataset
df = build_dataset(collective_scores) #only keeps videos with a tournesol score above 20

Save the dataframe

In [4]:
import datetime
df.to_csv('tournesol_scores_above_20_'+datetime.date.today().isoformat()+'.csv')
